# User Defined Rebound Collision Resolutions
In the `CloseEncounter` example, we discuss methods for resolving collisions in REBOUND through exceptions and the use of the `sim.collision_resolve = "merge"` method.

Using the same 3-Body setup, let's explore how to define and implement bespoke collision resolutions in python and pass them to the `sim.collision_resolve` method.

In [1]:
import rebound
import numpy as np

def setupSimulation():
    ''' Setup the 3-Body scenario'''
    sim = rebound.Simulation()
    sim.integrator = "ias15" # IAS15 is the default integrator, so we don't need this line
    sim.add(m=1.)
    sim.add(m=1e-3, a=1., r=np.sqrt(1e-3/3.)) # we now set collision radii!
    sim.add(m=5e-3, a=1.25, r=1.25*np.sqrt(5e-3/3.))
    sim.move_to_com()
    return sim

ModuleNotFoundError: No module named 'rebound'

To reitterate the previous method, let's run the built-in `merge` collision resolution method

In [8]:
sim = setupSimulation()
sim.collision = "direct"
sim.collision_resolve = "merge" # Built in function

print("Particles in the simulation at t=%6.1f: %d"%(sim.t,sim.N))
sim.integrate(100.)
print("Particles in the simulation at t=%6.1f: %d"%(sim.t,sim.N))

Particles in the simulation at t=   0.0: 3
Particles in the simulation at t= 100.0: 2


We can also use the built-in collision detection and apply our own function to resolve the collision. By default, if we don't set the sim.collision function pointer, `REBOUND` will raise a `Collision` exception when a collision occurs, which we can catch. 

An indirect way of checking which particles collided is to check which ones have a `lastcollision` time equal to the current simulation time.

In [9]:
sim = setupSimulation()
sim.collision = "direct"
# we don't set sim.collision_resolve this time

try:
    sim.integrate(100.)
except rebound.Collision:
    collided = []
    for p in sim.particles:
        if p.lastcollision == sim.t:
            collided.append(p.index)
    # Custom resolution

print("Particles {0} collided".format(collided))

Particles [1, 2] collided
